In [1]:
import pandas as pd

In [8]:
df = pd.read_excel('../WOT_internal_refined_6.xlsx').drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

## Remove unanswerable

In [9]:
df = df[df['is_unanswerable'] == False]

## Split datata based on Conversation

In [61]:
from sklearn.model_selection import train_test_split

df_conversations = df.groupby('conversation').count().reset_index()[['conversation','question']].rename(columns={'question':'count'})
df_conversations.loc[32, 'count'] = 6

conversations = list(df_conversations['conversation'])
count = list(df_conversations['count'])

conversation_train, conversation_test, count_train, _ = train_test_split(conversations, count,
                                                               test_size=0.5, random_state = 30, stratify=count)

conversation_train, conversations_val, _, _ = train_test_split(conversation_train, count_train, 
                                                               test_size=0.4, random_state= 42, stratify=count_train)

df['data_split'] = df['conversation'].apply(lambda x: 'train' if x in conversation_train else 'test' if x in conversation_test else 'validation')
df.groupby('data_split').count().reset_index()

,data_split,conversation,question,domain,document_url,answer_orig_wot,history,parsed_html,title,steps,...,is_causal,is_listing,is_history,is_navigation,is_unanswerable,answers_extr,context,input,output,random_number_split
0,test,412,412,412,412,412,375,412,412,412,...,412,412,412,412,412,412,412,412,412,412
1,train,248,248,248,248,248,225,248,248,248,...,248,248,248,248,248,248,248,248,248,248
2,validation,167,167,167,167,167,154,167,167,167,...,167,167,167,167,167,167,167,167,167,167


In [62]:
print(0.5 * len(df)) 
print(0.3 * len(df)) 
print(0.2 * len(df)) 

413.5
248.1
165.4


In [63]:
df.to_excel('../WOT_extractive_final.xlsx')